為了運行以下的筆記本，如果你還未設定，請在 .env 檔案內設置 openai key 為 `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

接住，我哋會將嵌入索引載入到一個 Pandas Dataframe。嵌入索引儲存在一個叫做 `embedding_index_3m.json` 嘅 JSON 檔案入面。嵌入索引包含咗截至 2023 年 10 月底每個 YouTube 逐字稿嘅嵌入。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

接住落嚟，我哋會建立一個叫做 `get_videos` 嘅函數，用嚟喺 Embedding Index 入面搜尋同查詢最相關嘅影片。呢個函數會返回同查詢最相似嘅頭5條影片。函數嘅運作方式如下：

1. 首先，會複製一份 Embedding Index。
2. 之後，會用 OpenAI Embedding API 計算查詢嘅 Embedding。
3. 跟住會喺 Embedding Index 入面新增一個叫做 `similarity` 嘅新欄位。`similarity` 呢個欄位會存放查詢嘅 Embedding 同每個影片片段嘅 Embedding 之間嘅 cosine similarity。
4. 之後，會根據 `similarity` 呢個欄位去篩選 Embedding Index，只保留 cosine similarity 大過或等於 0.75 嘅影片。
5. 最後，會根據 `similarity` 呢個欄位排序 Embedding Index，並返回頭5條影片。


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

這個函數非常簡單，只是輸出搜尋查詢的結果。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. 首先，Embedding Index 會被載入到一個 Pandas Dataframe。
2. 之後，系統會提示用戶輸入查詢內容。
3. 然後會呼叫 `get_videos` 函數，根據查詢內容在 Embedding Index 進行搜尋。
4. 最後會呼叫 `display_results` 函數，將搜尋結果顯示給用戶。
5. 系統會再次提示用戶輸入查詢內容。這個流程會一直重複，直到用戶輸入 `exit` 為止。

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.hk.png)

系統會提示你輸入查詢內容。請輸入你的查詢並按下 Enter。應用程式會返回一個與查詢相關的影片清單，並提供一個連結，直接跳到影片中回答該問題的位置。

你可以試下以下這些查詢：

- 乜嘢係 Azure Machine Learning？
- 卷積神經網絡（convolutional neural networks）點運作？
- 乜嘢係神經網絡？
- 我可唔可以用 Jupyter Notebooks 配合 Azure Machine Learning？
- 乜嘢係 ONNX？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人工翻譯。本翻譯所引致的任何誤解或曲解，我們概不負責。
